In [1]:
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    NestedField,
    StringType,
    DoubleType,
    ListType,
    StructType,
    LongType,
    IntegerType,
)

In [2]:
osm_data_schema = Schema(
    NestedField(
        field_id=1, name="business_id", field_type=StringType(), optional=False
    ),
)

In [3]:
catalog = load_catalog(
    "nessie",
    **{
        "uri": "http://172.16.100.64:19120/iceberg/main/",
        "s3.endpoint": "http://172.16.100.64:9000",
        "s3.access-key-id": "ZXPByJUzucJ7Y9OkwUhl",
        "s3.secret-access-key": "OSE2YujEDOCvdZR5Gi6pXarB6zja5pvPW6QRHTgl",
    },
)

In [4]:
table_test_name = f"scraping.osm_test_data"

In [5]:
catalog

nessie (<class 'pyiceberg.catalog.rest.RestCatalog'>)

In [6]:
import pyarrow as pa
import pandas as pd

table = catalog.create_table_if_not_exists(table_test_name, schema=osm_data_schema)
all_mapper_data = ["test"]
mapper_ddf = pa.Table.from_pandas(
    pd.DataFrame(all_mapper_data, columns=["business_id"])
)
table.append(mapper_ddf)

In [7]:
table_scan = table.scan()

In [8]:
df = table_scan.to_pandas()
df

,business_id
0,test


In [9]:
catalog.drop_table(table_test_name)

In [10]:
from pyiceberg.exceptions import NoSuchTableError

try:
    table.refresh()
except NoSuchTableError:
    print("delete success")

delete success


In [11]:
sample_7_data_path = "/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven/road_info/5.pkl"

In [12]:
import pandas as pd

sample_7_data = pd.read_pickle(sample_7_data_path)
sample_7_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   road_geometry                  100 non-null    geometry
 1   road_length                    100 non-null    float64 
 2   road_length_service            100 non-null    float64 
 3   road_length_residential        100 non-null    float64 
 4   road_length_tertiary           100 non-null    float64 
 5   road_length_secondary          100 non-null    float64 
 6   road_length_unclassified       100 non-null    float64 
 7   road_length_motorway_link      100 non-null    float64 
 8   road_length_motorway           100 non-null    float64 
 9   road_length_motorway_junction  100 non-null    float64 
 10  road_length_services           100 non-null    float64 
 11  road_length_footway            100 non-null    float64 
 12  road_length_secondary_link   

In [13]:
sample_7_data_geometry = sample_7_data.road_geometry[0]
type(sample_7_data_geometry)

shapely.geometry.polygon.Polygon

In [14]:
from shapely.geometry import Polygon


def polygon_to_list(polygon):
    """
    Converts a Shapely Polygon object to a list of coordinates.

    Args:
        polygon: A Shapely Polygon object.

    Returns:
        A list of coordinates, where each coordinate is a tuple of (x, y) floats.
    """
    coords = list(polygon.exterior.coords)
    # Remove the duplicate first and last coordinate
    return coords[:-1]


# Example usage:
polygon = Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])
coordinates = polygon_to_list(polygon)
print(coordinates)  # Output: [(0.0, 0.0), (1.0, 0.0), (1.0, 1.0), (0.0, 1.0)]
converted_polygon = Polygon(coordinates)
print(converted_polygon == polygon)

[(0.0, 0.0), (1.0, 0.0), (1.0, 1.0), (0.0, 1.0)]
True


In [15]:
coverted_sample_7_data_geometry = Polygon(polygon_to_list(sample_7_data_geometry))
sample_7_data_geometry == coverted_sample_7_data_geometry

np.True_

In [16]:
# for column "road_geometry" apply polygon_to_list
sample_7_data["road_geometry"] = sample_7_data["road_geometry"].apply(
    lambda x: polygon_to_list(x)
)
sample_7_data["building_geometry"] = sample_7_data["building_geometry"].apply(
    lambda x: polygon_to_list(x)
)

In [17]:
sample_7_data.road_geometry

0     [(11197920.0, 1536047.0), (11197920.0, 1535547...
1     [(11198420.0, 1536047.0), (11198420.0, 1535547...
2     [(11198920.0, 1536047.0), (11198920.0, 1535547...
3     [(11199420.0, 1536047.0), (11199420.0, 1535547...
4     [(11199920.0, 1536047.0), (11199920.0, 1535547...
                            ...                        
95    [(11200420.0, 1540547.0), (11200420.0, 1540047...
96    [(11200920.0, 1540547.0), (11200920.0, 1540047...
97    [(11201420.0, 1540547.0), (11201420.0, 1540047...
98    [(11201920.0, 1540547.0), (11201920.0, 1540047...
99    [(11202420.0, 1540547.0), (11202420.0, 1540047...
Name: road_geometry, Length: 100, dtype: object

In [18]:
sample_7_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   road_geometry                  100 non-null    object 
 1   road_length                    100 non-null    float64
 2   road_length_service            100 non-null    float64
 3   road_length_residential        100 non-null    float64
 4   road_length_tertiary           100 non-null    float64
 5   road_length_secondary          100 non-null    float64
 6   road_length_unclassified       100 non-null    float64
 7   road_length_motorway_link      100 non-null    float64
 8   road_length_motorway           100 non-null    float64
 9   road_length_motorway_junction  100 non-null    float64
 10  road_length_services           100 non-null    float64
 11  road_length_footway            100 non-null    float64
 12  road_length_secondary_link     100 non-null

In [8]:
import os

folder_path = "/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data"
file_name_list = os.listdir(folder_path)
file_path_list = [f"{folder_path}/{file_name}" for file_name in file_name_list]

In [9]:
print(len(file_path_list))
file_path_list[:3]

1568649


['/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data/075f256f701504d8de4268b249879b552e66b3b802e032a749ece82b85aa5c43.pkl',
 '/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data/8504bbab22ed9f88b687207ccde5335d585ce9a578a535257f8d3337d6f7404f.pkl',
 '/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data/3c1976e31fbb3e13b002ccafb9aabaf024e4a1a2badcc029c546fda282fd387b.pkl']

In [23]:
import pandas as pd

file_path = file_path_list[130]
df = pd.read_pickle(file_path)

In [24]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 102 entries, ('way', np.int64(27464545)) to ('way', np.int64(1219467493))
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   geometry                  102 non-null    geometry
 1   building                  102 non-null    object  
 2   name                      33 non-null     object  
 3   opening_hours             8 non-null      object  
 4   shop                      8 non-null      object  
 5   website                   12 non-null     object  
 6   wikidata                  1 non-null      object  
 7   addr:street               10 non-null     object  
 8   amenity                   8 non-null      object  
 9   name:en                   7 non-null      object  
 10  fee                       1 non-null      object  
 11  operator                  1 non-null      object  
 12  parking                   1 non-null      obje

In [25]:
df

geometry  \
element id                                                              
way     27464545    POLYGON ((11230993.826 1453575.665, 11231004.7...   
        27487186    POLYGON ((11230873.379 1453741.015, 11230872.4...   
        27727077    POLYGON ((11231183.47 1453475.469, 11231187.92...   
        153538101   POLYGON ((11230915.357 1453314.473, 11230917.5...   
        153538106   POLYGON ((11230887.561 1453257.442, 11230947.8...   
...                                                               ...   
        1212165125  POLYGON ((11230779.592 1453379.18, 11230779.94...   
        1212205557  POLYGON ((11230869.917 1453539.502, 11230870.1...   
        1219467289  POLYGON ((11230874.503 1453241.691, 11230862.2...   
        1219467492  POLYGON ((11230911.06 1453297.385, 11230919.32...   
        1219467493  POLYGON ((11230970.037 1453362.138, 11230992.3...   

                   building                  name      opening_hours  shop  \
element id                                                                   
way     27464545        yes        Central Marina  Mo-Su 11:00-23:00  mall   
        27487186        yes            ตลาดหงตุลา                NaN   NaN   
        27727077        yes                   NaN                NaN   NaN   
        153538101       yes                   NaN                NaN   NaN   
        153538106       yes  Alcazar Cabaret Show                NaN   NaN   
...                     ...                   ...                ...   ...   
        1212165125     roof                   NaN                NaN   NaN   
        1212205557     roof                   NaN                NaN   NaN   
        1219467289     roof                   NaN                NaN   NaN   
        1219467492      yes                   NaN                NaN   NaN   
        1219467493      yes                   NaN                NaN   NaN   

                                                              website  \
element id                                                              
way     27464545    http://www.centralplaza.co.th/en/branch-detail...   
        27487186                                                  NaN   
        27727077                                                  NaN   
        153538101                                                 NaN   
        153538106                    https://www.alcazarthailand.com/   
...                                                               ...   
        1212165125                                                NaN   
        1212205557                                                NaN   
        1219467289                                                NaN   
        1219467492                                                NaN   
        1219467493                                                NaN   

                    wikidata  addr:street      amenity  \
element id                                               
way     27464545    Q5060537          NaN          NaN   
        27487186         NaN  พัทยาสายสอง  marketplace   
        27727077         NaN          NaN      parking   
        153538101        NaN          NaN          NaN   
        153538106        NaN          NaN      theatre   
...                      ...          ...          ...   
        1212165125       NaN          NaN          NaN   
        1212205557       NaN          NaN          NaN   
        1219467289       NaN          NaN          NaN   
        1219467492       NaN          NaN          NaN   
        1219467493       NaN          NaN          NaN   

                                    name:en  ... check_date description  \
element id                                   ...                          
way     27464545                        NaN  ...        NaN         NaN   
        27487186    Hong Tu La Night Market  ...        NaN         NaN   
        27727077                        NaN  ...        NaN         NaN   
        15353810

In [1]:
df.info()

NameError: name 'df' is not defined

In [1]:
import os

dir_path = os.getcwd()
file_directory = f"{dir_path}/data/2024_11_data_7_eleven_poi_data/poi_data"  # Replace with your file directory


all_files = [os.path.join(file_directory, f) for f in os.listdir(file_directory)]

In [2]:
all_files[:3]

['/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data/075f256f701504d8de4268b249879b552e66b3b802e032a749ece82b85aa5c43.pkl',
 '/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data/8504bbab22ed9f88b687207ccde5335d585ce9a578a535257f8d3337d6f7404f.pkl',
 '/home/thanatorn/coding/cro_location_intelligence/extract_poi_data/data/2024_11_data_7_eleven_poi_data/poi_data/3c1976e31fbb3e13b002ccafb9aabaf024e4a1a2badcc029c546fda282fd387b.pkl']

In [3]:
len("075f256f701504d8de4268b249879b552e66b3b802e032a749ece82b85aa5c43")

64